<a href="https://colab.research.google.com/github/bb426/PyTorch-tutorials-kr/blob/master/torch_tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
N, D_in, H, D_out = 64, 1000, 100, 10

In [ ]:
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

In [ ]:
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

In [ ]:
learning_rate = 1e-6

In [ ]:
for t in range(10):
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    loss = np.square(y_pred - y).sum()
    print(f'{t} epoch, {loss:.0f}')

    # backprop
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # update
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 epoch, 211
1 epoch, 199
2 epoch, 187
3 epoch, 175
4 epoch, 165
5 epoch, 155
6 epoch, 146
7 epoch, 137
8 epoch, 129
9 epoch, 121


In [ ]:
import torch

dtype = torch.float
device = torch.device('cuda')

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(10):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    loss = (y_pred - y).pow(2).sum().item()
    print(f'{t} epoch, {loss:.0f}')

    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # update
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 epoch, 33364364
1 epoch, 31433824
2 epoch, 33470514
3 epoch, 33996788
4 epoch, 29262458
5 epoch, 20299608
6 epoch, 11580825
7 epoch, 5996296
8 epoch, 3186441
9 epoch, 1896194


In [ ]:
import torch
torch.device('cuda')

device(type='cuda')

In [ ]:
# autograd

import torch
dtype = torch.float
device = torch.device('cuda')

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for i in range(10):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()

    print(f'{t} epoch, {loss:.0f}')
    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

9 epoch, 31170508
9 epoch, 23608798
9 epoch, 18951146
9 epoch, 14871194
9 epoch, 11050088
9 epoch, 7809904
9 epoch, 5345988
9 epoch, 3635232
9 epoch, 2504482
9 epoch, 1774824


In [ ]:
# autograd with a forward function

import torch

class MyReLU(torch.autograd.Function):
    """
    Built custom autograd function
    """
    @staticmethod
    def forward(input):
        """
        input tensor를 받아 출력 tensor return
        """
        ctx.save_for_backward(input)
        # relu
        return input.clamp(min=0) 

    @staticmethod
    def backward(ctx, grad_output):
        """
        gradients를 return
        """
        input, = ctx.saved_tensor()
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device('cpu')

for i in range(10):
    relu = MyReLU.apply

    y_pred = relu(x.mm(w1)).mm(w2)

    loss = (y_pred - y).pow(2).sum()

    print(f'{i} epoch, {loss:.0f}')
    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()
    

In [ ]:
# backprop using nn Module

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)


model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H, bias=False),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out, bias=False)
)

loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-3

for i in range(10):
    y_pred = model(x)  ## nn 모듈은 입력 tensor를 모듈에 직접 전달하여 출력 tensor 생성할 수 있음 (__call__ 연산자를 override 해서 그렇다고 함)
    loss = loss_fn(y_pred, y)

    print(f'{i} epoch, {loss:.0f}')

    # 먼저 gradient 초기화
    model.zero_grad()
    
    # gradient 계산
    loss.backward()

    # update
    with torch.no_grad():
        for param in model.parameters(): #model.paramemters()에 있는 parameters는 모두 tensor이므로 gradient에 접근 가능함
            param -= learning_rate * param.grad



0 epoch, 1
1 epoch, 1
2 epoch, 1
3 epoch, 1
4 epoch, 1
5 epoch, 1
6 epoch, 1
7 epoch, 1
8 epoch, 1
9 epoch, 1


In [ ]:
# optim package

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in range(10):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    print(f'{i} epoch, {loss:.0f}')

    optimizer.zero_grad() #gradients 초기화, loss.backward()는 덮어쓰는게 아니라 += 형태라 반드시 초기화해줘야 함
    loss.backward() #gradient 계산
    optimizer.step() #update


0 epoch, 658
1 epoch, 641
2 epoch, 624
3 epoch, 608
4 epoch, 592
5 epoch, 576
6 epoch, 561
7 epoch, 546
8 epoch, 532
9 epoch, 518


In [ ]:
# nn.Module
# Sequencial 보다 더 복잡한 모델을 구성해야 할 경우

class TwoLayerNet(torch.nn.Module):
    # 새 모듈을 정의
    def __init__(self, D_in, H, D_out):
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

model = TwoLayerNet(D_in, H, D_out)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for i in range(10):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    print(i, ' th iter', loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



0  th iter 667.561279296875
1  th iter 617.8712158203125
2  th iter 574.3640747070312
3  th iter 536.1549072265625
4  th iter 502.1356506347656
5  th iter 471.53076171875
6  th iter 443.6327209472656
7  th iter 418.0291748046875
8  th iter 394.4185791015625
9  th iter 372.456787109375
